In [0]:
import pyspark.sql.functions as F

CATALOG = "catalogo_progetto"
SILVER  = "silver"
GOLD    = "gold"

# Cambia se i nomi differiscono nel tuo dataset
COL_BOOKING_ID   = "booking_id"
COL_CUSTOMER_ID  = "customer_id"
COL_ROOM_ID      = "room_id"
COL_HOTEL_ID     = "hotel_id"
COL_TOTAL        = "total_amount"
COL_AMOUNT       = "amount"
COL_SOURCE       = "source"
COL_STATUS       = "status"          # es. 'confirmed' / 'cancelled'
COL_CHECKIN      = "checkin_date"
COL_CHECKOUT     = "checkout_date"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{GOLD}")
spark.sql(f"USE CATALOG {CATALOG}")


In [0]:
bks = spark.table(f"{CATALOG}.{SILVER}.bookings")
pmt = spark.table(f"{CATALOG}.{SILVER}.payments")
cus = spark.table(f"{CATALOG}.{SILVER}.customers")
rms = spark.table(f"{CATALOG}.{SILVER}.rooms")
htl = spark.table(f"{CATALOG}.{SILVER}.hotels")


In [0]:
daily_rev = (
    bks
    .filter( (F.col(COL_STATUS) == "confirmed") & F.col(COL_TOTAL).isNotNull() )
    .groupBy(F.col(COL_CHECKIN).alias("date"))     # o usa la data giusta di competenza
    .agg(
        F.sum(COL_TOTAL).alias("gross_revenue"),
        F.countDistinct(COL_BOOKING_ID).alias("bookings_count")
    )
    .orderBy("date")
)

daily_rev.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.daily_revenue")


In [0]:
by_source = (
    bks.groupBy(COL_SOURCE)
       .agg(
           F.count("*").alias("total_bookings"),
           F.sum(F.when(F.col(COL_STATUS) == "cancelled", 1).otherwise(0)).alias("cancelled")
       )
       .withColumn("cancellation_rate_pct", F.round(F.col("cancelled")/F.col("total_bookings")*100, 2))
       .orderBy("cancellation_rate_pct", ascending=False)
)

by_source.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.cancellation_rate_by_source")


In [0]:
# Controllo veloce (opzionale)
print("bks:", bks.columns)
print("pmt:", pmt.columns)

# Filtri “validi”
payments_valid = pmt.filter(
    (~F.col("dq_orphan_payment")) &
    (~F.col("dq_invalid_currency")) &
    (~F.col("dq_over_amount")) &
    F.col(COL_AMOUNT).isNotNull()
)

bookings_valid = bks.filter(F.col(COL_TOTAL).isNotNull())

# 1) Portiamo hotel_id nei pagamenti tramite booking_id
pmt_with_hotel = (
    payments_valid
    .join(bookings_valid.select(COL_BOOKING_ID, COL_HOTEL_ID), on=COL_BOOKING_ID, how="inner")
)

# 2) Somme per hotel
payments_by_hotel = (
    pmt_with_hotel.groupBy(COL_HOTEL_ID)
    .agg(F.sum(COL_AMOUNT).alias("total_payments_value"))
)

bookings_by_hotel = (
    bookings_valid.groupBy(COL_HOTEL_ID)
    .agg(F.sum(COL_TOTAL).alias("total_bookings_value"))
)

# 3) Join finale + collection_rate
coll = (
    bookings_by_hotel
    .join(payments_by_hotel, on=COL_HOTEL_ID, how="left")
    .fillna({"total_payments_value": 0.0})
    .withColumn(
        "collection_rate",
        F.when(F.col("total_bookings_value") > 0,
               F.round(F.col("total_payments_value")/F.col("total_bookings_value"), 4))
         .otherwise(F.lit(None))
    )
)

coll.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.collection_rate")



In [0]:
b1 = bks.select(
    F.col(COL_BOOKING_ID).alias("booking_id_1"),
    F.col(COL_ROOM_ID).alias("room_id"),
    F.col(COL_CHECKIN).alias("checkin_1"),
    F.col(COL_CHECKOUT).alias("checkout_1")
)
b2 = bks.select(
    F.col(COL_BOOKING_ID).alias("booking_id_2"),
    F.col(COL_ROOM_ID).alias("room_id"),
    F.col(COL_CHECKIN).alias("checkin_2"),
    F.col(COL_CHECKOUT).alias("checkout_2")
)

overlap = (
    b1.join(b2, on="room_id")
      .filter(F.col("booking_id_1") < F.col("booking_id_2"))
      .withColumn("overlap_start", F.greatest("checkin_1", "checkin_2"))
      .withColumn("overlap_end",   F.least("checkout_1","checkout_2"))
      .filter(F.col("overlap_start") < F.col("overlap_end"))
      .select("room_id","booking_id_1","booking_id_2","overlap_start","overlap_end")
)

overlap.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.overbooking_alerts")


In [0]:
cust_val = (
    bks.filter(F.col(COL_TOTAL).isNotNull())
       # .filter(F.col(COL_STATUS) == "confirmed")   # sblocca se vuoi solo confermate
       .groupBy(COL_CUSTOMER_ID)
       .agg(
           F.countDistinct(COL_BOOKING_ID).alias("bookings_count"),
           F.sum(COL_TOTAL).alias("revenue_sum"),
           F.round(F.avg(COL_TOTAL), 2).alias("avg_ticket")
       )
)

cust_val.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.customer_value")
